# Apache Toree Demo

## Simple Spark Scala

Test notebook with simple Spark Scala code.

Take numbers 1 to 100, keep the numbers that are even, square them, and keep the first 10.

In [1]:
sc.parallelize(1 to 100).
  filter(x => x % 2 == 0).
  map(x => x * x).
  take(10)

Array(4, 16, 36, 64, 100, 144, 196, 256, 324, 400)

Use tab for auto-complete.

## Test CSV Library

### Useful functions

Define some functions.

In [2]:
// Grab URL contents
def getUrl(url:String):String = 
  scala.io.Source.fromURL(url).mkString

// Write file
def fileWrite(path:String,contents:String) = {
  import java.io.{PrintWriter,File}
  val writer = new PrintWriter(new File(path))
  writer.write(contents)
  writer.close
}

### Download Prices

Get the historical stock price of AAPL and save it in AAPL.csv

In [3]:
val symbol = "AAPL"
val baseUrl = "http://real-chart.finance.yahoo.com"
val url = s"${baseUrl}/table.csv?s=${symbol}&g=d&ignore=.csv"
val csv = getUrl(url)
val csvFile = s"${symbol}.csv"
fileWrite(csvFile, csv)
println(csvFile)

AAPL.csv


### Highest Prices

Find the days with the highest adjusted close prices.

In [4]:
val stockRdd = sc.textFile(csvFile).
  filter(line => line matches ".*\\d.*").
  map(line => line.split(",")).
  map(fields => (fields(6).toDouble,fields(0))).
  sortBy({case (close,date) => close},false)

stockRdd.take(5).foreach(println)

(130.67132,2015-05-22)
(130.579411,2015-02-23)
(130.235775,2015-04-27)
(130.20796,2015-07-20)
(130.178369,2015-05-27)


### Load CSV

Now lets use SQL to analyze the stock instead of directly manipulating records.

Load CSV file as data frame.

In [5]:
val df = sqlContext.read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferSchema", "true").
    load("AAPL.csv")

### View Data Frame

What does `df` look like?

In [6]:
df.select("Date","Adj Close").show

+----------+----------+
|      Date| Adj Close|
+----------+----------+
|2016-03-18|105.919998|
|2016-03-17|105.800003|
|2016-03-16|105.970001|
|2016-03-15|104.580002|
|2016-03-14|102.519997|
|2016-03-11|102.260002|
|2016-03-10|101.169998|
|2016-03-09|101.120003|
|2016-03-08|101.029999|
|2016-03-07|101.870003|
|2016-03-04|103.010002|
|2016-03-03|     101.5|
|2016-03-02|    100.75|
|2016-03-01|100.529999|
|2016-02-29| 96.690002|
|2016-02-26| 96.910004|
|2016-02-25| 96.760002|
|2016-02-24| 96.099998|
|2016-02-23| 94.690002|
|2016-02-22| 96.879997|
+----------+----------+
only showing top 20 rows



### SQL Queries

Register it as a SQL table.

In [7]:
df.registerTempTable("aapl")

Find out how many rows it has.

In [8]:
sqlContext.sql("SELECT COUNT(1) AS row_count FROM aapl").show

+---------+
|row_count|
+---------+
|     8893|
+---------+



### Highest Prices

Find out what the highest adjusted close was.

In [9]:
sqlContext.sql("SELECT MAX(`Adj Close`) AS max_close FROM aapl").show

+---------+
|max_close|
+---------+
|130.67132|
+---------+



Find the dates of the 5 highest adjusted close prices.

In [10]:
sqlContext.sql("""SELECT Date,`Adj Close` FROM aapl 
    ORDER BY `Adj Close` DESC LIMIT 5""").show

+----------+----------+
|      Date| Adj Close|
+----------+----------+
|2015-05-22| 130.67132|
|2015-02-23|130.579411|
|2015-04-27|130.235775|
|2015-07-20| 130.20796|
|2015-05-27|130.178369|
+----------+----------+

